# Setup

In [ ]:
!pip install google-adk

In [12]:
from google.cloud import aiplatform
from google import genai
from google.genai import types
from google.adk.agents import LlmAgent
from vertexai.preview.reasoning_engines import AdkApp

In [2]:
GOOGLE_CLOUD_PROJECT = "luciexu"
GOOGLE_CLOUD_LOCATION = "europe-west4"
MODEL = "gemini-2.0-flash"

doc_uri = "gs://luciexu-tmp/Capgemini- RFP MVP Solution TDD v2.pdf"

In [3]:
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_LOCATION)

# Tools

In [4]:
DISCOVERY_TOOL_SYSTEM_PROMPT = """
You are an AI assistant tasked with helping users. When answering, adhere to the following guildelines:
**Accuracy:** Ensure your answers are factually correct and grounded in the documents provided as a list of uris.
**Detail:** Provide comprehensive and informative answers, elaborating on key concepts and providing context. Be detailed and return an exhaustive answer.
**Language:** Strictly identify the language of ther user query and always repond in the same language regardless of the document language.
Now look at these documents and answer the user query. 
"""

DISCOVERY_TOOL_QUERY_PROMPT = "Inspect the file provided as a GCS uri and generate detailed recommendations to improve the overall security posture, with a focus on SAIF compliance."

def analysis_tool(gcs_uri: str):
    """Analyze a document provided as a GCS uri.

    Args:
        gcs_uri (str): Document GCS uri
    """

    contents = [
        types.Part.from_text(text=DISCOVERY_TOOL_QUERY_PROMPT),
        types.Part.from_uri(
            file_uri=gcs_uri,
            mime_type="application/pdf",
        )
    ]

    client = genai.Client(
        vertexai=True, project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_LOCATION,
    )
    response = client.models.generate_content(
        model=MODEL,
        contents=contents,
        config=types.GenerateContentConfig(
            system_instruction=DISCOVERY_TOOL_SYSTEM_PROMPT,
            max_output_tokens=1000,
            temperature=0.3,
        ),
    )
    return response



In [5]:
analysis_tool(doc_uri)

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.1924254632020213,
      content=Content(
        parts=[
          Part(
            text="""Basado en la información proporcionada en el documento, aquí hay recomendaciones detalladas para mejorar la postura general de seguridad, con un enfoque en el cumplimiento de SAIF:

1.  **Data Access Management (DAM):**
    *   **Implementar una estructura de carpetas común:** Establecer una estructura de carpetas consistente para mejorar la escalabilidad y el mantenimiento. La estructura actual se basa en la codificación rígida de rutas de archivos específicos, lo que dificulta la gestión.
    *   **Automatizar la gestión de ACLs:** Alinear el enfoque de las Listas de Control de Acceso (ACLs) con las mejores prácticas acordadas por Datacloud para automatizar y escalar la gestión de permisos.
    *   **Centralizar la gestión de metadatos:** Actualizar y mantener la información d

# Agent

In [16]:
# Model params
model = MODEL

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]

generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.1,
   max_output_tokens=1000,
   top_p=0.95,
)

In [ ]:
# Agent params
agent = LlmAgent(
   model=model,
   name='SAIFGuard',
   description="SAIFGuard is an agent that helps analyze security documents.",
   generate_content_config=generate_content_config, 
   tools=[analysis_tool] # Add other tools here
)
app = AdkApp(agent=agent)

In [11]:
async for event in app.async_stream_query(
    user_id="TEST_USER_ID",
    message=f"Help me analyze my document at {doc_uri}",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-c7620318-addf-49c4-b4fa-e5e20508a0c5', 'args': {'gcs_uri': 'gs://luciexu-tmp/Capgemini- RFP MVP Solution TDD v2.pdf'}, 'name': 'analysis_tool'}}], 'role': 'model'}, 'finish_reason': <FinishReason.STOP: 'STOP'>, 'usage_metadata': {'candidates_token_count': 26, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 26}], 'prompt_token_count': 79, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 79}], 'total_token_count': 105, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-f43166ec-1a58-4193-b479-bacc816067eb', 'author': 'SAIFGuard', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': '2ebd3c8b-777d-4876-8aa6-9debd062c2fd', 'timestamp': 1755775890.973001}
{'content': {'parts': [{'function_response': {'id': 'adk-c7620318-addf-49c4-b4fa-e5e20508a0c5', 'name': 'analysis_tool',

In [23]:
"/n".join([part["text"] for part in event["content"]["parts"]])

"Okay, I have analyzed the document. Here are the detailed recommendations to improve the overall security posture of the GenAI MVP solution, with a focus on SAIF (Security Architecture Implementation Framework) compliance, based on the provided Technical Design Document (TDD):\n\n**1. Data Access Management and Security:**\n\n*   **Scalable ACL Solution (SAIF Principle: Least Privilege):** The current ACL implementation relies on hard-coding folder paths in `config.py` and manual maintenance. This is not scalable or maintainable. Recommendation: Align the ACL approach with Datacloud's best practices. Conduct a workshop to design a scalable solution for ACLs, potentially leveraging structured data/metadata related to opportunities, SBUs, and BLs. SAIF Benefit: Ensures that users only have access to the data they need, reducing the risk of unauthorized access and data breaches.\n*   **Common Folder Structure (SAIF Principle: Standardization):** The document mentions the lack of a common